In [1]:
using PyPlot,JLD2,Statistics
# rc("font",family="serif")
# include("CGS.jl")

### Plot of simulated years observed vs TTVs

In [2]:
include("decompose_ttvs.jl")
include("plot_likelihood.jl")
include("corner_plot.jl")
include("plot_ttvs.jl")

plot_ttvs (generic function with 2 methods)

In [ ]:
fit = jldopen("OUTPUTS/p3_fittry003params.jld2","r")

In [ ]:
@load ("OUTPUTS/p3_fittry003params.jld2")

In [ ]:
years = [4,6,8,10,12,14,16,18,20,24,28,32,36,40]
function calc_jd2(nyrs)
return    (nyrs * 365.25) + jd1
end

In [ ]:
for i=1:length(years)
    println("NYears: ",years[i]," ; jd2 = ",calc_jd2(years[i]))
end

In [ ]:
pname = ["mu_1","P_1","t01","e1 cos(om1)","e1 sin(om1)",
            "mu_2","P_2","t02","e2 cos(om2)","e2 sin(om2)",
            "mu_3","P_3","t03","e3 cos(om3)","e3 sin(om3)",
            "tmax sin(phi0)","tmax cos(phi0)","deltaphi"]

In [ ]:
function vectorize_data(include_moon::Bool=false)
    names = ["mu", "P", "t0", "e cos(om)", "e sin(om)"]
    planet = ["venus", "earth", "jupiter"]
    nparam = length(names)*length(planet)+1
#     state_total = zeros((nsteps-iburn)*nwalkers+50,nparam-1)
    for iparam=1:nparam-1
#     state_total[:,iparam] += vec(par_mcmc[:,iburn:nsteps,iparam])
    if iparam < nparam
        pname = string(planet[div(iparam-1,5)+1]," ",names[mod(iparam-1,5)+1])
    end
        println("index: ",iparam, " ",pname," : ",pbest_global[iparam])
    end
#     for iparam=1:3
#     if iparam < nparam
end

In [ ]:
function extract_data(filename)
	f = jldopen(String(filename), "r")
	return f
end
#unsure if this works within for loops
#might have used in 200000 step moon run on desktop

In [ ]:
f = extract_data("OUTPUTS/moon_fitmtry3params.jld2")

In [ ]:
@load("mcmc_mtry3results.jld2")
@load("OUTPUTS/moon_fitmtry3params.jld2")

In [ ]:
include("corner_plot.jl")

## Posterior Distributions

### Plot masses of Venus and Earth in $M_{\oplus}$

In [ ]:
xvalue = vec(par_mcmc[:,iburn:nsteps,1]).* CGS.MSUN/CGS.MEARTH
yvalue = vec(par_mcmc[:,iburn:nsteps,6]).*CGS.MSUN/CGS.MEARTH
optx = mu_1.* CGS.MSUN/CGS.MEARTH
opty = mu_2.* CGS.MSUN/CGS.MEARTH
truex = 0.815
truey = 1
nbins = 50
corner_plot(xvalue,yvalue,nbins,optx,opty,truex,truey)
xlabel(L"Mass of Venus [$M_{\oplus}$]")
ylabel(L"Mass of Earth [$M_{\oplus}$]")
# savefig("massposteriors.png")

In [6]:
include("corner_plot.jl")
corner_hist(10.0,20.0,"EMB","p3",50)

In [5]:
corner_hist(10.0,20.0,"noEMB","p3",50)

### Plot eccentricities of Venus and Earth

In [ ]:
xvalue = sqrt.(vec(par_mcmc[:,iburn:nsteps,4]).^2 + vec(par_mcmc[:,iburn:nsteps,5]).^2)
yvalue = sqrt.(vec(par_mcmc[:,iburn:nsteps,9]).^2 + vec(par_mcmc[:,iburn:nsteps,10]).^2)
optx = sqrt(e1cosw1^2 + e1sinw1^2)
opty = sqrt(e2cosw2^2 + e2sinw2^2)
truex = 0.00677323
truey = 0.01671022
nbins=50
corner_plot(xvalue,yvalue,nbins,optx,opty,truex,truey)
xlabel("Eccentricity of Venus")
ylabel("Eccentricity of Earth")
# savefig("eccposteriors.png")

### Plot periods of Venus and Earth?

In [ ]:
xvalue = vec(par_mcmc[:,iburn:nsteps,2])
yvalue = vec(par_mcmc[:,iburn:nsteps,7]) 
optx = P_1
opty = P_2
truex = 225
truey = 365
nbins=50
corner_plot(xvalue, yvalue, nbins, optx, opty, truex, truey)
xlabel(L"Period of Venus [$days$]")
ylabel(L"Period of Earth [$days$]")


In [ ]:
10 .^ range(log10(10.6),stop=log10(14.2),length=200)

### Plot mass of Jupiter $M_{Jup}$ vs its eccentricity

In [ ]:
xvalue = vec(par_mcmc[:,iburn:nsteps,11]).* CGS.MSUN/CGS.MJUPITER
yvalue = sqrt.(vec(par_mcmc[:,iburn:nsteps,14]).^2 + vec(par_mcmc[:,iburn:nsteps,15]).^2)
optx = mu_3.* CGS.MSUN/CGS.MJUPITER
opty = sqrt(e3cosw3^2 + e3sinw3^2)
truex = 1
truey = 0.04839266
nbins=50
corner_plot(xvalue, yvalue, nbins, optx, opty, truex, truey)
xlabel(L"Mass of Jupiter [$M_{Jup}$]")
ylabel("Eccentricity of Jupiter")
# savefig("jupiterposteriors.png")

### Plot time of inferior conjuction [JD - 24550000] $t_{03}$ vs Jupiter period $P_3$. 

In [ ]:
xvalue = vec(par_mcmc[:,iburn:nsteps,12])
yvalue = vec(par_mcmc[:,iburn:nsteps,13]) .- 2.443e6
optx = P_3
opty = t03 .- 2.443e6
# truex = 1
# truey = 0.04839266
nbins=50
corner_plot(xvalue, yvalue, nbins, optx, opty, truex, truey)
xlabel(L"Period of Jupiter [$days$]")
ylabel(L"$t_{0}$ of Jupiter")

### Plot TTV of Earth due to Moon $t_{max}$ vs phase change of Moon per Earth year $\Delta \phi$

In [ ]:
# function plot_posterior(samples,bins,range)
# tmax = vec(sqrt(par_mcmc[:,iburn:nsteps,16]).^2 + vec(par_mcmc[:,iburn:nsteps,17]).^2)
xvalue = sqrt.(vec(par_mcmc[:,iburn:nsteps,16].^2) + vec(par_mcmc[:,iburn:nsteps,17].^2)).* (24 * 60)
yvalue = vec(par_mcmc[:,iburn:nsteps,18]).* 57.2957795 
optx = sqrt(param[16]^2 + param[17]^2).* (24 * 60)
opty = param[18].* 57.2957795 
# truex = 
truey = 2.3122 .* 57.2957795 
meanx=mean(xvalue);sigmax=std(xvalue)
meany=mean(yvalue);sigmay=std(yvalue)
nbins=50
corner_plot(xvalue,yvalue,nbins,optx,opty,truex,truey)
xlabel("Amplitude of Moon [min]")
ylabel("Phase Change of Moon [deg]")
# savefig("moonposteriors.png")

### Plot longitude of ascending node $\Omega$ vs longitude of pericenter $\bar \omega$; with argument of pericenter $\omega$

In [ ]:
mu_1,P_1,t01,e1cosw1,e1sinw1,
mu_2,P_2,t02,e2cosw2,e2sinw2,
mu_3,P_3,t03,e3cosw3,e3sinw3,
tmaxsinphi0,tmaxcosphi0,deltaphi = f["pbest_global"]

In [ ]:
calc_deg(value) = value * 180/pi
calc_omega(pomega,Omega) = pomega - Omega

In [ ]:
function calc_L(t,t0,P,esinw)
    lambda = (360/P .* (t.-t0)) .+ 2*esinw
    return lambda
end

In [ ]:
P_1/365.25,P_2/365.25,P_3/365.25

In [ ]:
e1 = sqrt.(e1cosw1.^2 + e1sinw1.^2)
e2 = sqrt.(e2cosw2.^2 + e2sinw2.^2)
e3 = sqrt.(e3cosw3.^2 + e3sinw3.^2)
e1,e2,e3

In [ ]:
w1 = calc_deg(atan(e1sinw1,e1cosw1))
w2 = calc_deg(atan(e2sinw2,e2cosw2))
w3 = calc_deg(atan(e3sinw3,e3cosw3))
w1,w2,w3

Agol & Deck 2015 equation 5 for mean longitude

$L \equiv \lambda$. Check to see if $\lambda_{E} \sim \lambda_{V}$ at their respective transit times. $\lambda mod 2 \pi = \lambda_{obs}$[what?] where $\lambda_{obs}$ is longitude that both Earth and Venus have.

In [ ]:
L_1 = calc_L(t01,J2000,P_1,e1sinw1)
L_2 = calc_L(t02,J2000,P_2,e2sinw2)
L_3 = calc_L(t03,J2000,P_3,e3sinw3)
L_1,L_2,L_3

In [ ]:
L_1+w1,L_2+w2,L_3+w3

Venus transit occurs 0.277 radians after it goes through periastron. Earth transit occurs -0.552 radians after it goes through periastron

In [ ]:
J2000 = 2451545.0
((J2000 - t01)/365.25)

In [ ]:
true_w1 = 131.60246718 - 0.00268329*.5
true_w2 = 102.94719 - 0.32327364*.5
true_w3 = 14.72847983 - 0.21252668*.5
true_w1,true_w2,true_w3

## 10 second noise

In [ ]:
f = extract_data("OUTPUTS/moon_fitmtry2params.jld2")

In [ ]:
@load("mcmc_mtryresults.jld2")

In [ ]:
mu_1,P_1,t01,e1cosw1,e1sinw1,
mu_2,P_2,t02,e2cosw2,e2sinw2,
mu_3,P_3,t03,e3cosw3,e3sinw3,
tmaxsinphi0,tmaxcosphi0,deltaphi = f["pbest_global"]

In [ ]:
w1 = calc_deg(atan(e1sinw1,e1cosw1))
w2 = calc_deg(atan(e2sinw2,e2cosw2))
w3 = calc_deg(atan(e3sinw3,e3cosw3))
w1,w2,w3

In [ ]:
L_1 = calc_L(tt1,tt01,P_1,e1sinw1)
L_2 = calc_L(tt2,tt02,P_2,e2sinw2)
L_1,L_2

In [ ]:
(((360/P_1) .* (tt01 .- J2000)) .+ 181.97909950)

In [ ]:
((360/P_2) .* (tt02 .- J2000) .+ 100.46457166)

In [ ]:
[(360/P_3) .* (t03 .- J2000) .+ 34.39644051,
(360/P_3) .* (t03 .- J2000) .+ 34.39644051 + 360*1,
(360/P_3) .* (t03 .- J2000) .+ 34.39644051 + 360*2]

In [ ]:
77.56936298207381+w1,77.76976911511983+w2,77.91649744476126+w3

In [ ]:
xvalue = calc_deg.(atan.(vec(par_mcmc[:,iburn:nsteps,5]),vec(par_mcmc[:,iburn:nsteps,4]))) .+ 77.6
yvalue = calc_deg.(atan.(vec(par_mcmc[:,iburn:nsteps,10]),vec(par_mcmc[:,iburn:nsteps,9]))) .+ 77.8
optx = calc_deg(atan(e1sinw1,e1cosw1)) + 77.6
opty = calc_deg(atan(e2sinw2,e2cosw2)) + 77.8
truex = 131.601125535
truey = 102.78555318000001
meanx=mean(xvalue);sigmax=std(xvalue)
meany=mean(yvalue);sigmay=std(yvalue)
nbins=50
corner_plot(xvalue,yvalue,nbins,optx,opty,truex,truey)
xlabel(L"Longitude $\varpi$ of Venus [$deg$]")
ylabel(L"Longitude $\varpi$ of Earth [$deg$]")

In [ ]:
xvalue = calc_deg.(atan.(vec(par_mcmc[:,iburn:nsteps,15]),vec(par_mcmc[:,iburn:nsteps,14]))) .+ 77.9
yvalue = calc_deg.(atan.(vec(par_mcmc[:,iburn:nsteps,10]),vec(par_mcmc[:,iburn:nsteps,9]))) .+ 77.8
optx = calc_deg(atan(e3sinw3,e3cosw3)) + 77.9
opty = calc_deg(atan(e2sinw2,e2cosw2)) + 77.8
truex = 14.62221649
truey = 102.78555318000001
nbins=50
corner_plot(xvalue,yvalue,nbins,optx,opty,truex,truey)
xlabel(L"Longitude $\varpi$ of Jupiter [$deg$]")
ylabel(L"Longitude $\varpi$ of Earth [$deg$]")

In [ ]:
calc_deg(2pi)

In [ ]:
(360/P_3 * (t03 - J2000) - 34.39644051)

In [ ]:
include("testplot.jl")

In [ ]:
corner_planet(vec(par_mcmc[:,iburn:nsteps,6]).* CGS.MSUN/CGS.MEARTH,
    vec(par_mcmc[:,iburn:nsteps,9]),
    vec(par_mcmc[:,iburn:nsteps,10]),
    vec(par_mcmc[:,iburn:nsteps,7]),
0.815,-0.004,-0.002,224.5)

In [ ]:
corner_planet(vec(par_mcmc[:,iburn:nsteps,11]).* CGS.MSUN/CGS.MEARTH,
    vec(par_mcmc[:,iburn:nsteps,14]),
    vec(par_mcmc[:,iburn:nsteps,15]),
    vec(par_mcmc[:,iburn:nsteps,12])
318,-0.004,-0.002,224.5)

In [3]:
include("MCMC.jl")
include("sim_times.jl")
include("fit_moon.jl")

fit_moon (generic function with 4 methods)

In [11]:
function moon_run(label,sigma)
sim_times(jd1,jd2,jdsize,true,sigma,false)
datafile = string("INPUTS/tt_data",sigma,"snoEMB.txt")
fit_moon(datafile,label,jd1,jd2,jdsize,p3in,p3out,np3,nphase,dpin,dpout,ndp,true,sigma,false)
end

function moon_mcmc(label,nwalkers,nsteps)
fitfile = string("OUTPUTS/moon_fit",label,"params.jld2")
foutput = string("NYR40/moon_",label)
f = jldopen(String(fitfile),"r")
MCMC(f["pbest_global"],f["lprob_best"],foutput,nsteps,nwalkers,f["nplanet"],f["ntrans"],f["tt0"],f["tt"],f["sigtt"],false,true)
end

moon_mcmc (generic function with 1 method)

In [5]:
###### NYR40 ########
jd1 = 2.4332825e6
jd2 = 2.4478925e6
jdsize = 1000
p3in = 4230.0
p3out = 4430.0
np3 = 100
nphase = 36
dpin = 0.0 
dpout = 2*pi
ndp = 72
# nwalkers = 50
# nsteps = 1000

@time moon_run("try1",15.0)

Initial parameters: [3.0e-6, 224.7007709658875, 2.4335037688751635e6, 0.01, 0.01, 3.0e-6, 365.2564691537294, 2.4336244074187805e6, 0.01, 0.01]
Initial chi-square: 5753.449556375698
Finished 2-planet fit: [3.827644325612551e-6, 224.70077111011403, 2.4335037688856414e6, -0.025372832865190646, -0.056331799746939706, 2.8906042187669944e-6, 365.2564666098448, 2.4336244074206473e6, -0.006450363929878842, -0.043148344639483326]
Period: 4229.999999999999 chi: -367.7580780015665 Param: [3.80406321772746e-6, 224.700770849545, 2.4335037688489705e6, -0.045565704826349904, -0.08094982653548963, 2.751004894346839e-6, 365.25646869365744, 2.433624407238144e6, -0.02189280571470263, -0.06248227379154274, 0.00035056209521912827, 4229.999999999999, 2.4398186693543782e6, -0.16288148762407814, 0.09323587438982874]
Period: 4231.9743576935625 chi: -367.63359827065653 Param: [3.8050302434117534e-6, 224.7007708335396, 2.4335037688496145e6, -0.045473137730080465, -0.08095314258598825, 2.7515015833971112e-6, 365.

Period: 4271.655606951618 chi: -365.6432811104242 Param: [3.815906117169158e-6, 224.70077051577567, 2.4335037688629474e6, -0.04433248107519834, -0.08104630920863473, 2.7581168214661504e-6, 365.25646658902355, 2.4336244072873746e6, -0.020863006393280342, -0.0624989262197737, 0.0003747753796420451, 4271.655606951618, 2.4398046681385036e6, -0.14672663165026437, 0.09137620274657643]
Period: 4273.649407450871 chi: -365.5705538961015 Param: [3.814701573442763e-6, 224.70077050006282, 2.433503768863664e6, -0.044426586211429954, -0.08102286503709885, 2.7577872118171263e-6, 365.25646649222085, 2.433624407289784e6, -0.02093598396847858, -0.062477337221269226, 0.0003760977229891732, 4273.649407450871, 2.444078977479898e6, -0.14563144029945582, 0.09163335248154693]
Period: 4275.644138558953 chi: -365.5005518502823 Param: [3.81758473549983e-6, 224.70077048570883, 2.4335037688642913e6, -0.04414578860502746, -0.08095631727689738, 2.759263171880352e-6, 365.2564663852445, 2.433624407292312e6, -0.0207090

Period: 4315.7348589794265 chi: -364.70481352654696 Param: [3.817366531647957e-6, 224.70077018476283, 2.4335037688780488e6, -0.04385905544287356, -0.08107089193694265, 2.7617610581438405e-6, 365.25646437604206, 2.43362440734203e6, -0.02042182190769477, -0.062473251814117406, 0.00040002685637958383, 4315.7348589794265, 2.4441127206745506e6, -0.12999052510946257, 0.09106618466408492]
Period: 4317.749233523683 chi: -364.6966965434417 Param: [3.818897109571879e-6, 224.700770171526, 2.433503768878684e6, -0.04369827626416348, -0.08102492502083611, 2.762691246920611e-6, 365.2564642761077, 2.433624407344549e6, -0.020290544368845656, -0.06243722422540385, 0.00040115418858184104, 4317.749233523683, 2.444113690241461e6, -0.1293447763934424, 0.09090922965857794]
Period: 4319.764548279731 chi: -364.69173385966394 Param: [3.818699589290627e-6, 224.7007701574232, 2.433503768879373e6, -0.043686822555983544, -0.08100860931982985, 2.7628378172443757e-6, 365.2564641779471, 2.4336244073470533e6, -0.020278

Period: 4360.268965199171 chi: -365.2660338623707 Param: [3.813439376004138e-6, 224.70076988946605, 2.4335037688926496e6, -0.04371954545169556, -0.08100455401963068, 2.7639086822793323e-6, 365.25646226747403, 2.4336244073965843e6, -0.020240866749642785, -0.06239197730192606, 0.00042286947313759294, 4360.268965199171, 2.4441518672250453e6, -0.11545290095347086, 0.09119751044378868]
Period: 4362.304126092185 chi: -365.3288676052799 Param: [3.811521292034022e-6, 224.70076987668247, 2.4335037688933206e6, -0.04386337574061766, -0.08098010117415481, 2.763355564895889e-6, 365.2564621770166, 2.433624407399012e6, -0.020351178976598033, -0.06237044275024632, 0.00042406226591539876, 4362.304126092185, 2.444154477582764e6, -0.11462577344695003, 0.09133992611770425]
Period: 4364.340236899043 chi: -365.3949474662087 Param: [3.812936015075541e-6, 224.70076986530998, 2.433503768893924e6, -0.04370826329555344, -0.08098248721154586, 2.764122736756297e-6, 365.25646208406016, 2.433624407401462e6, -0.02022

Period: 4405.262619255287 chi: -367.38765815123844 Param: [3.802529648653023e-6, 224.70076964003385, 2.4335037689060518e6, -0.044095031634274875, -0.08083410947688811, 2.7634827579109936e-6, 365.25646035725225, 2.4336244074482806e6, -0.02047192241086333, -0.06223932491437083, 0.00044203932824975694, 4405.262619255287, 2.444193043494326e6, -0.10322510120507072, 0.09155641548951977]
Period: 4407.3187809915735 chi: -367.5198309586942 Param: [3.801697326048026e-6, 224.70076962995782, 2.4335037689066236e6, -0.04412995265224188, -0.08086824206656959, 2.763393469826662e-6, 365.2564602777733, 2.4336244074505046e6, -0.020497334054045663, -0.062264842015715915, 0.0004425913064908664, 4407.3187809915735, 2.444195199872244e6, -0.10282382065225083, 0.09166419932677836]
Period: 4409.375902443875 chi: -367.65496233972846 Param: [3.7992165408105507e-6, 224.7007696187733, 2.4335037689072415e6, -0.044340620585149475, -0.0808805106600048, 2.7622765704214414e-6, 365.25646020094524, 2.433624407452679e6, -0

deltaphi: 0.5309734062405285 chi: -379.3113613250751 Param: [3.843051405100068e-6, 224.70077068638955, 2.433503768854665e6, -0.04903471042287089, -0.07551031699259614, 2.7162434742623992e-6, 365.2564670617915, 2.433624407300779e6, -0.02557837177031613, -0.05899563488971626, -4.520068852472589e-6, 4328.01355930873, 2.454299265504693e6, 23.80054622487899, 1.4450595785760696, -0.0001695492520286771, 0.0001635721993589391, 0.5309734062405285]
deltaphi: 0.6194689739472832 chi: -365.04875235349823 Param: [3.7338323588162196e-6, 224.70077000009553, 2.433503768895224e6, -0.052948465573776894, -0.08378462183736783, 2.7173789397837557e-6, 365.25646408916975, 2.433624407361277e6, -0.027529221081329727, -0.06465504111718379, 0.0003935363886629655, 4350.714034449071, 2.4442285968896393e6, -0.08913956870955428, 0.11160783197162791, -0.0001412039501793671, 0.00031128296854894034, 0.6194689739472832]
deltaphi: 0.7079645416540379 chi: -379.4596978883182 Param: [4.810582789663161e-6, 224.70077097923973,

deltaphi: 2.2123891926688684 chi: -444.02520079977785 Param: [2.526656452329664e-6, 224.70077098633192, 2.4335037688517487e6, 0.002861432970218971, 0.0007352965380792142, 3.0437930507424112e-6, 365.2564621471861, 2.4336244074025345e6, 0.01504031533986775, 0.002960568112961731, 0.0009417437370194743, 4309.187016015247, 2.4433885045562196e6, 0.024404481606895973, -0.037724609302383166, 9.317476195724792e-5, -0.0016402935801090896, 2.2123891926688684]
deltaphi: 2.3008847603756233 chi: -324.47515063876745 Param: [2.5263919239907936e-6, 224.7007709864346, 2.4335037688517547e6, 0.002843720202035819, 0.0007312356742074699, 3.0437134528267434e-6, 365.2564621466153, 2.4336244074025564e6, 0.015027275363341933, 0.002957753056487025, 0.0009421768259166461, 4309.2163391111835, 2.443388646958187e6, 0.024443244222036888, -0.03768733038922679, 9.301384777001182e-5, -0.0016403271358780376, 2.3008847603756233]
deltaphi: 2.3893803280823778 chi: -438.11162925964527 Param: [2.526583992903605e-6, 224.700770

deltaphi: 3.8938049790972085 chi: -438.11250834414614 Param: [2.526691645566014e-6, 224.7007709857632, 2.433503768851778e6, 0.002882311451990049, 0.0007131596832524784, 3.0439398639330755e-6, 365.25646214430657, 2.4336244074026034e6, 0.015057392132403287, 0.002942904225655163, 0.0009418202295997062, 4309.213325633664, 2.443388694834002e6, 0.024422904811185364, -0.03770795780779436, 9.287290915732482e-5, 0.001640289503562527, 3.8938049790972085]
deltaphi: 3.982300546803963 chi: -324.4536807522112 Param: [2.5274892458072607e-6, 224.70077098611281, 2.4335037688517426e6, 0.002865507714425133, 0.0007349604608839774, 3.0438012466225506e-6, 365.25646214457277, 2.433624407402584e6, 0.015044049098754148, 0.0029599047154356185, 0.0009412991083281077, 4309.2405202590035, 2.443388362033245e6, 0.02435199334211986, -0.03777905083269599, 9.303519338440511e-5, 0.001639966410504903, 3.982300546803963]
deltaphi: 4.070796114510718 chi: -444.0215573097394 Param: [2.5267218360367654e-6, 224.70077098591398,

deltaphi: 5.575220765525549 chi: -369.7685440405985 Param: [3.724106651576919e-6, 224.70076999493168, 2.4335037688955152e6, -0.053928627514139856, -0.08387465737349672, 2.7123041468119817e-6, 365.25646409784855, 2.4336244073612685e6, -0.028310645000455728, -0.06471272939814257, 0.0003930065059238021, 4350.860692034868, 2.4442346145537714e6, -0.08842085528061808, 0.112994069409928, -0.00014204637098743912, -0.00031242073424423523, 5.575220765525549]
deltaphi: 5.663716333232303 chi: -365.0743629624629 Param: [3.723724452814401e-6, 224.70076999560948, 2.433503768895524e6, -0.053971015940214015, -0.08381377315069118, 2.712137488998585e-6, 365.25646409805785, 2.433624407361269e6, -0.02834403283786813, -0.06466491369381418, 0.00039341450106830195, 4350.835339239996, 2.444234877548602e6, -0.0880412892092273, 0.11294160272519656, -0.00014186208214335966, -0.0003129126857652576, 5.663716333232303]
deltaphi: 5.7522119009390575 chi: -378.8696610564444 Param: [3.8828740692099155e-6, 224.7007706739

(-309.3160059720751, [2.5265647811793776e-6, 224.70077098564252, 2.43350376885178e6, 0.0028674076124366866, 0.0007089788163744732, 3.043850173258022e-6, 365.25646214388945, 2.433624407402613e6, 0.015045951341550911, 0.002939665797684039, 0.0009418229714197295, 4309.234648075649, 2.443388777280714e6, 0.0244013764146702, -0.03769740922107507, 9.283390193776704e-5, 0.0016402786597255067, 3.9708561373552547])

In [15]:
@time moon_mcmc("try1",50,1000)

InterruptException: InterruptException:

In [7]:
@time moon_run("try2",30.0)
# @time moon_mcmc("try1",50,3000)

Initial parameters: [3.0e-6, 224.70077139170323, 2.433503768850492e6, 0.08, -0.03, 3.0e-6, 365.25647179330895, 2.433624407337019e6, -0.09, 0.09]
Initial chi-square: 3156.714026200558
Finished 2-planet fit: [3.411565710722212e-6, 224.70077131451276, 2.4335037688694033e6, -0.21784884962641596, -0.23166367673335297, 2.6400966993854317e-6, 365.2564699253804, 2.433624407324508e6, -0.15779250594510544, -0.2129940637181811]
Period: 4229.999999999999 chi: -304.9522176883975 Param: [3.7869052675415737e-6, 224.70077125849178, 2.4335037688248893e6, -0.14518321825931993, -0.2682880412729101, 2.7504106022828964e-6, 365.2564718990988, 2.433624407142266e6, -0.08255712049339338, -0.24407316390099518, 0.0003716240050549698, 4229.999999999999, 2.4440857687694007e6, -0.3408341993705756, 0.21843179758146317]
Period: 4231.9743576935625 chi: -304.8583922223405 Param: [3.78798206617282e-6, 224.70077124280314, 2.433503768825537e6, -0.1448842666946278, -0.2684093079734769, 2.750877736113748e-6, 365.25647180554

Period: 4271.655606951618 chi: -303.37441851040785 Param: [3.803941850828527e-6, 224.70077094687625, 2.4335037688384484e6, -0.1402837328747381, -0.26907864470307086, 2.760133442965171e-6, 365.2564699104827, 2.4336244071898824e6, -0.07740848473897208, -0.2444640041779204, 0.00039578520571372845, 4271.655606951618, 2.4441035033709435e6, -0.3216635779962323, 0.2179956764893005]
Period: 4273.649407450871 chi: -303.3210297820319 Param: [3.8069302992829423e-6, 224.7007709349954, 2.433503768839013e6, -0.13949951624384785, -0.2691127531131805, 2.7617031551644725e-6, 365.2564698184004, 2.433624407192175e6, -0.07658553591036311, -0.244463316974516, 0.00039680564454406083, 4273.649407450871, 2.4398297781106476e6, -0.32108878491814336, 0.21743248073699162]
Period: 4275.644138558953 chi: -303.2697424914603 Param: [3.8061858105200677e-6, 224.70077091984325, 2.433503768839725e6, -0.1397137533264763, -0.26893296076293094, 2.7614944977798526e-6, 365.25646972045706, 2.433624407194623e6, -0.0767767301452

Period: 4315.7348589794265 chi: -302.7091847255295 Param: [3.816255546755844e-6, 224.7007706394083, 2.43350376885292e6, -0.13655809396251525, -0.2691083335076583, 2.768464878270776e-6, 365.2564677944395, 2.433624407243236e6, -0.07331178717034255, -0.24430313839863835, 0.00042027234735856717, 4315.7348589794265, 2.4441279436188573e6, -0.30045462175366, 0.2209764163077014]
Period: 4317.749233523683 chi: -302.70551564013607 Param: [3.818992635026425e-6, 224.70077062589326, 2.4335037688536234e6, -0.13574489463008946, -0.2691774744046193, 2.770086821588711e-6, 365.2564676904858, 2.433624407245911e6, -0.07246796439297488, -0.24433045391590272, 0.0004222347153068796, 4317.749233523683, 2.4441274754473604e6, -0.2990452444118009, 0.22016523672271718]
Period: 4319.764548279731 chi: -302.70426218664124 Param: [3.817711154307847e-6, 224.70077061160492, 2.4335037688542753e6, -0.13604680716681272, -0.26925286959960043, 2.7694578408266024e-6, 365.25646759874024, 2.4336244072482307e6, -0.0727794182349

Period: 4360.268965199171 chi: -303.2027302797233 Param: [3.8247240485470715e-6, 224.70077035945067, 2.433503768867055e6, -0.1333565858499838, -0.2692343996062787, 2.7757765529815594e-6, 365.25646573876764, 2.433624407297216e6, -0.0697613573032024, -0.24427278382794243, 0.0004421442469781784, 4360.268965199171, 2.4441548732425524e6, -0.28097391179423636, 0.2242636433106493]
Period: 4362.304126092185 chi: -303.25435502139476 Param: [3.822264372741683e-6, 224.70077034634804, 2.4335037688677222e6, -0.13396150923820654, -0.2692428880615449, 2.774642925354131e-6, 365.2564656497232, 2.4336244072995987e6, -0.07037644799980099, -0.24430794635750208, 0.00044274328246570577, 4362.304126092185, 2.4441575863417336e6, -0.2799889432220628, 0.22509949921777717]
Period: 4364.340236899043 chi: -303.308532733185 Param: [3.821012188412486e-6, 224.7007703340786, 2.433503768868399e6, -0.1343276231503518, -0.2689462978720144, 2.774255132402369e-6, 365.25646556075117, 2.4336244073020103e6, -0.070696939562821

Period: 4405.262619255287 chi: -304.9271471329761 Param: [3.823941078893179e-6, 224.70077012105452, 2.4335037688800646e6, -0.13242807927850894, -0.2687237647768555, 2.7793057794801306e-6, 365.25646386130677, 2.433624407348702e6, -0.06847704780220686, -0.2438320896400364, 0.0004606242369192348, 4405.262619255287, 2.444187248271683e6, -0.26214261138809597, 0.22952309063419246]
Period: 4407.3187809915735 chi: -305.03437673418097 Param: [3.826577853088598e-6, 224.70077011298508, 2.433503768880527e6, -0.1315840723460936, -0.26893982844220876, 2.780767035083014e-6, 365.25646377749, 2.43362440735098e6, -0.06761763868393683, -0.24397859649499376, 0.0004609112915475258, 4407.3187809915735, 2.4441869375951053e6, -0.2624494102853944, 0.22867437392166906]
Period: 4409.375902443875 chi: -305.14400275742435 Param: [3.822935431521623e-6, 224.7007701016478, 2.4335037688811203e6, -0.1325141505911413, -0.26885320842858557, 2.7790934019071446e-6, 365.2564637000953, 2.433624407353137e6, -0.068567346003469

deltaphi: 0.5309734062405285 chi: -312.79143860797467 Param: [3.0912518968656536e-6, 224.70077114248681, 2.433503768858196e6, -0.3221206538256917, -0.20946809353140322, 2.394862445796441e-6, 365.2564700743369, 2.4336244072209783e6, -0.25908186496551777, -0.19521750266003346, -0.00026625868862472475, 4373.008668891267, 2.4435401424708986e6, 0.5476626309368683, 0.1629851026578589, 0.00043576292657370445, 0.0006847905843484645, 0.5309734062405285]
deltaphi: 0.6194689739472832 chi: -304.0861133806882 Param: [3.726339888284431e-6, 224.70077043981752, 2.4335037688723565e6, -0.1649920405004762, -0.26875709135963255, 2.713723650875963e-6, 365.25646673809257, 2.433624407276505e6, -0.10260285532152011, -0.24518748243165697, 0.00039841083845795164, 4348.21869809837, 2.4442670049606846e6, -0.18805012555749892, 0.31558223125179435, -0.00021904878739138737, 0.00036066242508518747, 0.6194689739472832]
deltaphi: 0.7079645416540379 chi: -315.47742084661616 Param: [4.796682633455895e-6, 224.700771407516

deltaphi: 2.2123891926688684 chi: -371.2091277677754 Param: [2.5506084915418925e-6, 224.70077140728498, 2.4335037688277876e6, 0.05872302287808802, -0.005532250438232103, 3.036946122989083e-6, 365.2564657212822, 2.4336244072984727e6, 0.12282472106971937, 0.01879831418318545, 0.0009771250836669509, 4304.81695000085, 2.4434237037469177e6, 0.14091142054724867, -0.162689625013665, 6.227289213242072e-5, -0.0016114697842570841, 2.2123891926688684]
deltaphi: 2.3008847603756233 chi: -271.27384309090695 Param: [2.5516079263029367e-6, 224.70077141296315, 2.4335037688275753e6, 0.05802622866292538, -0.0015094834420639025, 3.036205066105014e-6, 365.256465731422, 2.4336244072983037e6, 0.12236556123946186, 0.020391545077802078, 0.0009785355689047287, 4304.843823129867, 2.4434217667583073e6, 0.14156402486421252, -0.16279962839634382, 6.360422458812232e-5, -0.0016114784621669663, 2.3008847603756233]
deltaphi: 2.3893803280823778 chi: -368.2564084053131 Param: [2.5546883840442016e-6, 224.7007714142021, 2.

deltaphi: 3.8938049790972085 chi: -368.2878776629629 Param: [2.552609847938338e-6, 224.70077141530348, 2.4335037688274775e6, 0.05791622049772429, -0.000261367335556501, 3.0360958347309453e-6, 365.25646574193826, 2.433624407298059e6, 0.12228920537136843, 0.020841930427158355, 0.000978324302393076, 4304.767195022306, 2.4434210156530226e6, 0.14153052770037974, -0.1630326827729802, 6.411521171664712e-5, 0.0016113133130271795, 3.8938049790972085]
deltaphi: 3.982300546803963 chi: -271.2528332575547 Param: [2.553514062384203e-6, 224.7007714109372, 2.4335037688276186e6, 0.05907876596716943, -0.002901026387264854, 3.036987401745047e-6, 365.25646572639204, 2.433624407298394e6, 0.12283796932993672, 0.01973922274873678, 0.0009755535164221794, 4304.826040576275, 2.443422195619545e6, 0.14033097222510194, -0.16347091810839207, 6.309798713356915e-5, 0.0016109483523499915, 3.982300546803963]
deltaphi: 4.070796114510718 chi: -371.1975074641757 Param: [2.5530450415138057e-6, 224.7007714116166, 2.43350376

deltaphi: 5.575220765525549 chi: -309.99569211806215 Param: [3.7320713714841897e-6, 224.70077043966376, 2.433503768872395e6, -0.16379888324851982, -0.2684219246933384, 2.716603937443992e-6, 365.2564667056186, 2.43362440727712e6, -0.10131582236974979, -0.24489198317630506, 0.00039951172134692405, 4348.472101749157, 2.444264971880454e6, -0.18760569630284465, 0.31502289884484663, -0.000219373639856188, -0.00036124514929729366, 5.575220765525549]
deltaphi: 5.663716333232303 chi: -304.1889239463656 Param: [3.7017771627569415e-6, 224.70077042777993, 2.4335037688731095e6, -0.17185757985546699, -0.2667534089922425, 2.701066064503232e-6, 365.25646674095725, 2.43362440727674e6, -0.10958671155275504, -0.24355172273654238, 0.00039828972303686906, 4348.524129453107, 2.4442815350252446e6, -0.18001551704705795, 0.322006903181265, -0.00021904956310259927, -0.00036594002737565815, 5.663716333232303]
deltaphi: 5.7522119009390575 chi: -315.5950475417292 Param: [3.840484499339429e-6, 224.70077096845154, 2

(-263.8916154128884, [2.550278002576836e-6, 224.70077140902484, 2.4335037688277382e6, 0.05832222873041434, -0.004445791285916213, 3.0366355089623276e-6, 365.2564657231458, 2.4336244072984695e6, 0.12261770877783908, 0.01926789685132212, 0.0009783151797975263, 4304.907433052951, 2.4434232821347327e6, 0.14145012931623513, -0.16247668291164935, 6.26657543896955e-5, 0.0016116240982533177, 3.9721666258096473])

In [ ]:
@time moon_mcmc("try2",50,3000)

In [14]:
println("test")

test
